# Préparation des données
Dans ce notebook, nous nettoyons les données et ajoutons quelques features de base.

In [1]:
import pandas as pd
import numpy as np
import pickle

## Table Transactions

In [5]:
transactions = pd.read_pickle("pickles/transactions.pkl")

In [7]:
# Transformation des doublons en champ "quantité"
transactions = transactions.groupby(
    transactions.columns.tolist(),
    as_index=False
).size()

transactions.rename({'size': 'quantity'}, axis = 1, inplace = True)

In [8]:
# Tri par date descendante
transactions = transactions.sort_values('t_dat', ascending = False)

In [9]:
# Ajout de la notion de n° de semaine
last_day = transactions['t_dat'].max()
transactions['week_number'] = (last_day - transactions['t_dat']).dt.days // 7

In [13]:
transactions = pd.read_pickle('pickles/transactions_with_week_number.pkl')

In [31]:
# Typage des valeurs
transactions[['sales_channel_id', 'quantity', 'week_number']] = transactions[['sales_channel_id', 'quantity', 'week_number']].astype('int16')
transactions['price'] = transactions['price'].astype('float32')

In [80]:
transactions.to_pickle('pickles/transactions_clean.pkl')
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,quantity,week_number
28813418,2020-09-22,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0898573003,0.033881,2,1,0
28793633,2020-09-22,53d5f95331b01525404c3cbb2da6a84e1173dccb979d28...,0752814021,0.033881,2,2,0
28793635,2020-09-22,53da4b44e81286ed175a46d8ffd5a2baf47843089dc03f...,0793506006,0.025407,1,1,0
28793636,2020-09-22,53da4b44e81286ed175a46d8ffd5a2baf47843089dc03f...,0802459001,0.008458,1,1,0
28793637,2020-09-22,53da4b44e81286ed175a46d8ffd5a2baf47843089dc03f...,0874169001,0.042356,1,1,0


## Table Cart

In [16]:
carts = transactions.groupby(["customer_id", "t_dat", "sales_channel_id"], as_index = False).agg(
    total = ("price", "sum"),
    unique_articles = ("article_id", "count"),
    total_articles = ("quantity", "sum"),
    content = ("article_id", lambda x: list(x))
)


In [17]:
carts = carts.sort_values(['customer_id', 't_dat'])

In [18]:
carts['day_of_week'] = pd.to_datetime(carts['t_dat']).dt.day_of_week
carts['day_of_year'] = pd.to_datetime(carts['t_dat']).dt.day_of_year

### Ajout de l'intervalle entre deux paniers

In [19]:
carts['day_from_origin'] = (carts['t_dat'] - carts['t_dat'].min()).dt.days

In [20]:
carts['days_from_previous'] = carts['day_from_origin'].diff()

In [21]:
carts['previous_customer_id']=""
carts['previous_customer_id'].iloc[1:] = carts['customer_id'].iloc[0:-1]
carts.loc[carts['customer_id'] != carts['previous_customer_id'], 'days_from_previous'] = 0

/home/treb/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [22]:
carts['days_from_previous'] = carts['days_from_previous'].fillna(0).astype(int)
carts = carts.drop('previous_customer_id', 1)

/tmp/ipykernel_7029/2697235294.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  carts = carts.drop('previous_customer_id', 1)


In [23]:
carts.to_pickle('pickles/carts.pkl')
carts.head()

,customer_id,t_dat,sales_channel_id,total,unique_articles,total_articles,content,day_of_week,day_of_year,day_from_origin,days_from_previous
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,1,0.110119,3,3,"[0627759010, 0625548001, 0176209023]",3,361,98,0
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,2,0.010153,1,1,[0697138006],3,122,224,126
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,2,0.050831,1,2,[0568601006],5,145,247,23
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,1,0.033864,2,2,"[0745232001, 0607642008]",3,206,308,61
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,2,0.044051,1,1,[0656719005],2,261,363,55


In [25]:
carts[['customer_id', 't_dat', 'total', 'unique_articles', 'total_articles', 'days_from_previous']].head(5)

,customer_id,t_dat,total,unique_articles,total_articles,days_from_previous
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,0.110119,3,3,0
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,0.010153,1,1,126
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,0.050831,1,2,23
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,0.033864,2,2,61
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,0.044051,1,1,55


## Table client

In [27]:
customers = pd.read_pickle('pickles/customers.pkl')

In [43]:
customers['club_member_status'].isna().sum()

6062

In [41]:
customers['club_member_status'].value_counts()

ACTIVE        1272491
PRE-CREATE      92960
LEFT CLUB         467
Name: club_member_status, dtype: int64

#### Nettoyage de la table

In [56]:

 ## fichier 'customer'
   ### Colonne FN
     #### Remplacement des NaN par 0
customers['FN'] = customers['FN'].fillna(0)

   ### Colonne Active
     #### Remplacement des NaN par 0
customers['Active'] = customers['Active'].fillna(0)

   ### Colonne Club member status
     #### Remplacement des NaN par NON ACTIVE
customers['club_member_status'] = customers['club_member_status'].fillna('NON ACTIVE')
customers[customers['club_member_status']=='NON ACTIVE'] 

   ### Colonne Fashion News Frequency
     #### Option 1: Remplacement NaN par None
customers['fashion_news_frequency'].fillna('None', inplace = True)
     #### Correction d'un label écrit avec deux casses différentes
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].replace({'NONE': 'None'})

   ### Colonne age
     #### Option 1: Remplacement NaN par None
#customers['age'] = customers['age'].fillna('NONE')
     #### Option 2: Remplacement par la moyenne
#customers['age'] = customers['age'].fillna(customers['age'].mean())
     #### Option 3: Remplacement par zéro
customers['age'] = customers['age'].fillna(0)


#### Ajout de features liées aux achats

In [58]:
carts = pd.read_pickle('pickles/carts.pkl')

carts_customers = carts.groupby('customer_id', as_index = False).agg(
    average_cart_articles = ("unique_articles", "mean"),
    average_cart_price = ("total", "mean"),
    total_carts = ('customer_id', 'count'),
    total_articles = ("total_articles", 'sum'),
    total_price = ("total", 'sum'),
)
carts_customers.head()

,customer_id,average_cart_articles,average_cart_price,total_carts,total_articles,total_price
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,1.900000,0.054393,10,21,0.543932
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,3.250000,0.100510,24,86,2.412237
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,2.142857,0.086646,7,18,0.606525
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,2.000000,0.060983,1,2,0.060983
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,2.166667,0.078282,6,13,0.469695


In [59]:
# Intervalle moyen entre chaque panier
carts_intervals = carts[carts['days_from_previous'] != 0].groupby('customer_id', as_index = False).agg(
    average_cart_interval = ('days_from_previous', 'mean')
)
carts_intervals['average_cart_interval'] = carts_intervals['average_cart_interval'] / 7
carts_intervals.head()

,customer_id,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,9.809524
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,4.259740
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,17.285714
3,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,19.142857
4,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,33.428571


In [60]:
# Ajout des features sur la table des clients.
customers = customers.merge(carts_customers, on = 'customer_id', how = 'left')
customers = customers.merge(carts_intervals, on = 'customer_id', how = 'left')

# Remplissage des valeurs manquantes suites aux jointures.
customers.fillna(0, inplace = True)

In [74]:
# Typer les champs entiers en int32.
customers[['FN', 'Active', 'age', 'total_carts', 'total_articles']] = customers[['FN', 'Active', 'age', 'total_carts', 'total_articles']].astype('int16')

# Typer les champs de catégories en "categorical"
customers[['club_member_status', 'fashion_news_frequency', 'postal_code']] = customers[['club_member_status', 'fashion_news_frequency', 'postal_code']].astype('category')

In [76]:
customers.to_pickle('pickles/customers_clean.pkl')
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,average_cart_articles,average_cart_price,total_carts,total_articles,total_price,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,None,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.900000,0.054393,10,21,0.543932,9.809524
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,ACTIVE,None,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,3.250000,0.100510,24,86,2.412237,4.259740
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,ACTIVE,None,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,2.142857,0.086646,7,18,0.606525,17.285714
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,0,ACTIVE,None,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...,2.000000,0.060983,1,2,0.060983,0.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,1,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...,2.166667,0.078282,6,13,0.469695,19.142857


In [77]:

customers.iloc[:, :7].head(5)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,None,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,ACTIVE,None,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,ACTIVE,None,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,0,ACTIVE,None,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,1,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [79]:
customers[['customer_id', 'average_cart_articles',
       'average_cart_price', 'total_carts', 'total_articles', 'total_price',
       'average_cart_interval']].head(5)

,customer_id,average_cart_articles,average_cart_price,total_carts,total_articles,total_price,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,1.900000,0.054393,10,21,0.543932,9.809524
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,3.250000,0.100510,24,86,2.412237,4.259740
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,2.142857,0.086646,7,18,0.606525,17.285714
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,2.000000,0.060983,1,2,0.060983,0.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,2.166667,0.078282,6,13,0.469695,19.142857


## Table Articles

In [27]:
articles = pd.read_pickle('pickles/articles.pkl')

#### Nettoyage de la table

In [28]:
# Étape 3: Gestion des NaN
articles['detail_desc'].fillna('', inplace = True)

#### Features issues des transactions

In [ ]:
transactions = pd.read_pickle('pickles/transactions_clean.pkl')

transaction_articles = transactions.groupby('article_id', as_index = False).agg(
    total_purchases = ('quantity', 'sum'),
    average_quantity = ('quantity', 'mean'),
    average_price = ('price', 'mean')
)

In [29]:
articles = articles.merge(transaction_articles, on = 'article_id', how = 'left')

# Suppression des articles pour lesquels aucune transaction n'existe.
articles.dropna(subset = ['total_purchases'], axis = 0, inplace = True)

#### Ajout d'une colonne has_image (True / False)

In [30]:
# Chargement de la liste des fichiers images
# (Obtenu par la commande `find -name  *.jpg > images.csv` sous linux) 

images = pd.read_csv('csv/images.csv')
images['article_id'] = images['file_path'].apply(
    lambda x: 
        x.split('/')[2].split('.jpg')[0]
).astype(str)
images = images.sort_values('article_id')
images.head()

,file_path,article_id
73653,./010/0108775015.jpg,0108775015
73652,./010/0108775044.jpg,0108775044
73651,./010/0108775051.jpg,0108775051
16811,./011/0110065001.jpg,0110065001
16810,./011/0110065002.jpg,0110065002


In [31]:
# Fusion des deux tableaux en Left join
article_images = articles.merge(images, on ="article_id", how = 'left')

In [ ]:
article_images['has_image'] = article_images['file_path'].isna().apply(lambda x: not x)
articles = article_images.drop("file_path", 1)

#### Finalisation

In [33]:
# Suppression des catégories au format numérique
columns_to_drop = ['product_type_no',
       'graphical_appearance_no',
       'colour_group_code',
       'perceived_colour_value_id',
       'perceived_colour_master_id',
       'department_no',
       'index_group_no',
       'garment_group_no']

articles.drop(columns = columns_to_drop, axis = 1, inplace = True)

# Typage des catégories 
categories = ['product_type_name', 'product_group_name', 
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name',
       'perceived_colour_master_name',
       'department_name', 'index_name',
       'index_group_name', 'section_name',
       'garment_group_name']

articles[categories] = articles[categories].astype('category')

# Typage des champs entiers
articles['total_purchases'] = articles['total_purchases'].astype('int32')

In [34]:
# Sauvegarde finale
articles.to_pickle('pickles/articles_clean.pkl')
articles.head()

,article_id,product_code,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,...,index_name,index_group_name,section_no,section_name,garment_group_name,detail_desc,total_purchases,average_quantity,average_price,has_image
0,0108775015,0108775,Strap top,Vest top,Garment Upper body,Solid,Black,Dark,Black,Jersey Basic,...,Ladieswear,Ladieswear,16,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,10841,1.438752,0.008109,True
1,0108775044,0108775,Strap top,Vest top,Garment Upper body,Solid,White,Light,White,Jersey Basic,...,Ladieswear,Ladieswear,16,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,7250,1.279562,0.008093,True
2,0108775051,0108775,Strap top (1),Vest top,Garment Upper body,Stripe,Off White,Dusty Light,White,Jersey Basic,...,Ladieswear,Ladieswear,16,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,215,1.214689,0.004982,True
3,0110065001,0110065,OP T-shirt (Idro),Bra,Underwear,Solid,Black,Dark,Black,Clean Lingerie,...,Lingeries/Tights,Ladieswear,61,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",1044,1.063136,0.020055,True
4,0110065002,0110065,OP T-shirt (Idro),Bra,Underwear,Solid,White,Light,White,Clean Lingerie,...,Lingeries/Tights,Ladieswear,61,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",539,1.118257,0.018087,True


In [35]:
articles.columns

Index(['article_id', 'product_code', 'prod_name', 'product_type_name',
       'product_group_name', 'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_code', 'index_name', 'index_group_name',
       'section_no', 'section_name', 'garment_group_name', 'detail_desc',
       'total_purchases', 'average_quantity', 'average_price', 'has_image'],
      dtype='object')

In [36]:
articles[['article_id', 'product_code', 'prod_name', 'product_type_name',
       'product_group_name', 'garment_group_name',
       'total_purchases', 'average_quantity', 'average_price', 'has_image']].head()

,article_id,product_code,prod_name,product_type_name,product_group_name,garment_group_name,total_purchases,average_quantity,average_price,has_image
0,0108775015,0108775,Strap top,Vest top,Garment Upper body,Jersey Basic,10841,1.438752,0.008109,True
1,0108775044,0108775,Strap top,Vest top,Garment Upper body,Jersey Basic,7250,1.279562,0.008093,True
2,0108775051,0108775,Strap top (1),Vest top,Garment Upper body,Jersey Basic,215,1.214689,0.004982,True
3,0110065001,0110065,OP T-shirt (Idro),Bra,Underwear,"Under-, Nightwear",1044,1.063136,0.020055,True
4,0110065002,0110065,OP T-shirt (Idro),Bra,Underwear,"Under-, Nightwear",539,1.118257,0.018087,True


## Nettoyage des tables

In [33]:
customers = pd.read_pickle('pickles/customers.pkl')
articles = pd.read_pickle('pickles/articles.pkl')

### Articles

La proportion de lignes du fichier article avec une valeur manquante est de: 0.4%


#### À valider - Suppression des 995 articles qui n'ont jamais été achetés

In [35]:
articles = articles.dropna(subset=['total_purchases'])

In [36]:
articles.to_pickle('pickles/articles_clean.pkl')

### Clients

In [8]:

 ## fichier 'customer'
   ### Colonne FN
     #### Inspection colonne  
customers['FN'].head(30)
     #### Remplacement des NaN par 0
customers['FN'] = customers['FN'].fillna(0)

   ### Colonne Active
     #### Inspection colonne 
customers['Active'].head(30)
     #### Remplacement des NaN par 0
customers['Active'] = customers['Active'].fillna(0)

   ### Colonne Club member status
     #### Inspection colonne 
customers['club_member_status'].head(30)
     #### Remplacement des NaN par NON ACTIVE
customers['club_member_status'] = customers['club_member_status'].fillna('NON ACTIVE')
customers[customers['club_member_status']=='NON ACTIVE'] 

   ### Colonne Fashion News Frequency
     #### Inspection colonne 
customers['fashion_news_frequency'].head(30)     
customers[customers['fashion_news_frequency'].isna()]  # Pour voir le lien entre cette colonne (avec modalités NaN) et les autres colonnes 
     #### Option 1: Remplacement NaN par None
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].fillna('NONE')
     #### Option 2: Suppression des lignes qui ont des valeurs manquantes dans age et fashions_news_freq
#customers[(customers['fashion_news_frequency'].isna()) &(customers['age'].isna())]  --> detecter les lignes qui ont val manquantes dans 2 colonnes
#customers = customers.dropna(axis = 0, how='all', subset=['fashion_news_frequency', 'age']) --> suppression des lignes
     #### Option 3 (suggestion de Jean) -> garder les lignes avec NaN car explication logique

   ### Colonne age
     #### Inspection de la colonne 
customers['age'].head(30)
customers[customers['age'].isna()]  # Pour voir le lien entre cette colonne (avec modalités NaN) et les autres colonnes
     #### Option 1: Remplacement NaN par None
#customers['age'] = customers['age'].fillna('NONE')
     #### Option 2: Remplacement par la moyenne
customers['age'] = customers['age'].fillna(customers['age'].mean())



In [10]:
customers.to_pickle('pickles/customers_clean.pkl')

## Jointure Transactions - Customers - Articles

In [3]:
transactions = pd.read_pickle("pickles/transactions")
customers = pd.read_pickle("pickles/customers")
articles = pd.read_pickle("pickles/articles")

transactions = transactions.sample(frac = 0.1)

In [4]:
transactions = transactions.merge(customers, on="customer_id", how="left")

In [4]:
transactions = transactions.merge(articles, on="article_id", how="left")

In [5]:
transactions.to_pickle("pickles/transactions_all_infos")

In [6]:
transactions.head()

,customer_id,article_id,t_dat,price,sales_channel_id,quantity,day_of_week,day_of_year,FN,Active,...,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,total_purchases,average_quantity,average_price,has_image
0,06164dfaa997b0d4c51e321d09825df738db72f40fef84...,678942001,2019-05-22,0.016932,2,1,2,142,1.0,1.0,...,Ladieswear,11.0,Womens Tailoring,1003.0,Knitwear,"Short-sleeved jumper in a soft, fine-knit visc...",11671.0,1.0,0.016313,True
1,3eeeec7264479f541c8a6036dfd0ba0722c127a20540d1...,867300002,2020-04-09,0.025407,2,1,3,100,0.0,0.0,...,Ladieswear,15.0,Womens Everyday Collection,1010.0,Blouses,Sleeveless top in softly draping satin. Double...,760.0,1.0,0.020976,True
2,4e1c50b8dc8d5080c4def498baf75e60dfe4152ce301c0...,399223001,2019-05-15,0.033881,1,1,2,135,1.0,1.0,...,Divided,57.0,Ladies Denim,1016.0,Trousers Denim,"Jeggings in washed, superstretch denim with a ...",19604.0,1.0,0.030976,True
3,f0bba28bce9d6b7cf2aaa9b6bbe67b9c5e5079a6ad2a78...,720125008,2019-09-10,0.033508,2,1,1,253,0.0,0.0,...,Sport,5.0,Ladies H&M Sport,1005.0,Jersey Fancy,Sports tights in fast-drying functional fabric...,1933.0,1.0,0.032000,True
4,34121391358e8057d31b10667e970a37579549540da8b1...,883492001,2020-03-01,0.016932,1,1,6,61,1.0,1.0,...,Divided,52.0,Divided Accessories,1019.0,Accessories,Belt in imitation leather with a metal buckle....,40.0,1.0,0.015773,True


## Jointure Carts - Customers

In [7]:
carts = pd.read_pickle("pickles/carts")
customers = pd.read_pickle("pickles/customers")

In [8]:
carts = carts.merge(customers, on="customer_id", how="left")

In [9]:
carts.to_pickle("pickles/carts_all_infos")

In [10]:
carts.head()

,customer_id,t_dat,sales_channel_id,total,unique_articles,total_articles_x,content,day_of_week,day_of_year,day_from_origin,...,club_member_status,fashion_news_frequency,age,postal_code,average_cart_articles,average_cart_price,total_carts,total_articles_y,total_price,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,1,0.110119,3,3,"[176209023, 625548001, 627759010]",3,361,98,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,2,0.010153,1,1,[697138006],3,122,224,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,2,0.050831,1,1,[568601006],5,145,247,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,1,0.033864,2,2,"[607642008, 745232001]",3,206,308,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,2,0.044051,1,1,[656719005],2,261,363,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
